In [1]:
import subprocess
import os
import glob
import dataclasses
from serialize import serialize_cli

In [2]:
@dataclasses.dataclass
class Config:
    cli = 'C:\\Soft\\cutechess-1.3.1-win64\\cutechess-cli.exe'
    
    tt_size = 16
    concurrency = 8
    nodes = 50000
    n_rounds = 400
    pgn_out = None
    openings = 'openings/openings-6ply-1000.pgn'
    
    rating_interval = None
    tc_str = '10+0.1'
    
    base_conf = 'saturn_latest'

In [22]:
list(range(10))[:4][::2]

[0, 2]

In [7]:
def run_test(net_folder, n_latest, config: Config, reserialize=False, net_step=1):
    pt_nets = glob.glob(f'{net_folder}/*.pt')
    pt_nets.sort(key=os.path.getmtime, reverse=True)
    pt_nets = pt_nets[::net_step][:n_latest]
    nnue_nets = []
    for pt_net in pt_nets:
        net_name = os.path.basename(pt_net).removesuffix('.pt')
        net_path = f'{net_folder}/{net_name}.nnue'
        if reserialize or not os.path.isfile(net_path):
            serialize_cli(pt_net, net_path)
        nnue_nets.append((net_name, net_path))
        
    args = [config.cli, '-tournament', 'gauntlet']
    args += ['-concurrency', f'{config.concurrency}']
    if config.pgn_out:
        args += ['-pgnout', config.pgn_out]
    args += ['-draw', 'movenumber=40', 'movecount=4', 'score=10']
    args += ['-resign', 'movecount=4', 'score=500']
    args += ['-each', f'tc={config.tc_str}', f'nodes={config.nodes}', 'proto=uci', f'option.Hash={config.tt_size}']
    args += ['-openings', f'file={config.openings}', 'policy=round']
    args += ['-repeat', '-games', '2', '-rounds', f'{config.n_rounds}']
    if config.rating_interval:
        args += ['-ratinginterval', f'{config.rating_interval}']
#     args += ['-resultformat', 'concise']

    args += ['-engine', f'conf={config.base_conf}', 'name=master']
    
    for name, path in nnue_nets:
        path = os.path.abspath(path)
        args += ['-engine', f'conf={config.base_conf}', f'name={name}', 
                 f'option.evalfile={path}']
    return args

In [73]:
process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
cli_out = open('cli.out', 'w')
results = {}
result_output = False
for line in process.stdout:
    line = line.decode('utf-8')
    cli_out.write(line.strip() + '\n')
    
    if n_players == 2 and line.startswith('Elo'):
        vals = line.split()
        elo, error = float(vals[2]), float(vals[4][:-1])
        results[net_name] = (-elo, error)
    
    if n_players > 2 and line.startswith('Rank'):
        result_output = True
        continue
    
    if n_players > 2 and result_output:
        vals = line.split()
        rank, name, elo, error = float(vals[0]), vals[1], float(vals[2]), int(vals[3])
        results[name] = (elo, error)
        if rank + 1 == n_players:
            result_output = False
cli_out.close()

In [74]:
results

{'eblan_all': (-190.8, 451.9)}

In [3]:
pt_nets = glob.glob('d6nnv3/*.pt')

In [4]:
pt_nets

['d6nnv3\\net_0.pt',
 'd6nnv3\\net_10.pt',
 'd6nnv3\\net_100.pt',
 'd6nnv3\\net_110.pt',
 'd6nnv3\\net_120.pt',
 'd6nnv3\\net_130.pt',
 'd6nnv3\\net_140.pt',
 'd6nnv3\\net_150.pt',
 'd6nnv3\\net_20.pt',
 'd6nnv3\\net_30.pt',
 'd6nnv3\\net_40.pt',
 'd6nnv3\\net_50.pt',
 'd6nnv3\\net_60.pt',
 'd6nnv3\\net_70.pt',
 'd6nnv3\\net_80.pt',
 'd6nnv3\\net_90.pt']

In [97]:
os.path.basename(pt_nets[0])

'net_19.pt'

In [5]:
config = Config()
config.tt_size = 16
config.nodes = 75000
config.tc_str = '3600'
config.rating_interval = 20
config.concurrency = 16
config.pgn_out = 'pgns/d6nnv2_75knodes.pgn'

In [10]:
print(' '.join(run_test('d6nnv3', 3, config, net_step=2)))

C:\Soft\cutechess-1.3.1-win64\cutechess-cli.exe -tournament gauntlet -concurrency 16 -pgnout pgns/d6nnv2_75knodes.pgn -draw movenumber=40 movecount=4 score=10 -resign movecount=4 score=500 -each tc=3600 nodes=75000 proto=uci option.Hash=16 -openings file=openings/openings-6ply-1000.pgn policy=round -repeat -games 2 -rounds 400 -ratinginterval 20 -engine conf=saturn_latest name=master -engine conf=saturn_latest name=net_190 option.evalfile=C:\Users\aitaz\Documents\saturn\trainer\d6nnv3\net_190.nnue -engine conf=saturn_latest name=net_170 option.evalfile=C:\Users\aitaz\Documents\saturn\trainer\d6nnv3\net_170.nnue -engine conf=saturn_latest name=net_150 option.evalfile=C:\Users\aitaz\Documents\saturn\trainer\d6nnv3\net_150.nnue


In [40]:
serialize_cli('d6nnv2_x2data_wdl/net_10.pt', 'd6nnv2_x2data_wdl/net_10.nnue')